---
title: "Pool Cross-section and Panel Data"
subtitle: "One year is no longer enough"
author: Fernando Rios-Avila
jupyter: nbstata
format: 
  revealjs: 
    slide-number: true
    width: 1600
    height: 900
    code-fold: true
    code-overflow: wrap
    echo: true
    css: styles.css 
    chalkboard: true  
---


## Pooling Data together: Cross-section and Panel Data

- Up to this point, we have cover the analysis of cross-section data. 
  - Many individuals at a single point in time.
- Towards the end of the semester, We will also cover the analysis of time series data.
  - A single individual across time.
- Today, we will cover the analysis of panel data and repeated crossection: Many individuals across time.
  
- This type of data, also known as longitudinal data, has advantages over crossection, as it provides more information that helps dealing with the unknown of $e$.

- And its often the only way to answer certain questions.
  
## Pooling independent crossections

- We first consider the case of independent crossections. 
  - We have access to surveys that may be collected regularly. (Household budget surveys)
  - We assume that individuals across this surveys are independent from each other (no panel structure).
- This scenario is typically used for increasing sample-sizes and thus power of analysis (*larger N smaller SE*)
- Only minor considerations are needed when analyzing this type of data.
  - We need to account for the fact Data comes from different years. This can be done by including year dummies.
  - May need to Standardize variables to make them comparable across years. (inflation adjustments, etc.)

## Example {.scrollable}

Lets use the data `fertil1` to estimate the changes in fertility rates across time. This data comes from the *General Social Survey*.


In [ ]:
frause fertil1, clear
regress kids educ age agesq black east northcen west farm othrural town smcity i.year, robust  

- This allow us to see how fertility rates have changed across time.
- One could even interact the year dummies with other variables to see how the effect of other variables have changed across time.


In [ ]:
frause cps78_85, clear
regress lwage i.year##c.(educ i.female) exper expersq union, robust cformat(%5.4f)

## Good old Friend: Chow test

- The Chow test can be used to test whether the coefficients of a regression model are the same across two groups. 
  - we have seen this test back when we were discussing dummy variables.
- We can also use this test to check if coefficients of a regression model are the same across two time periods. (Has the wage structure changed across time?)
  - This is the case of interest here.
- Not much changes with before. Although it can be a bit more tedious to code.

## Example {.scrollable}


In [ ]:
frause cps78_85, clear
regress lwage i.year##c.(educ i.female exper expersq i.union), robust

In [ ]:
*| code-fold: false
test 85.year#c.educ 85.year#1.female 85.year#c.exper   85.year#c.expersq 85.year#1.union

## Using Pool Crossection for Causal Inference

- One advantage of pooling crossection data is that it could to be used to estimate causal effects using a method known as Differences in Differences (DnD)

- Consider the following case:
  - There was a project regarding the construction of an incinerator in a city. You are asked to evaluate what the impact of this was on the prices of houses around the area. 
  - You have access to data for two years: 1978 and 1981.
  - In 1978, there was no information about the project. In 1981, the project was announced, but it only began operations in 1985.

##

- we could start estimating the project using the simple model:
$$rprice = \beta_0 + \beta_1 nearinc + e$$

using only 1981 data. But this would not be a good idea. Why?


In [ ]:
*| classes: larger
frause kielmc, clear
regress rprice nearinc if year == 1981, robust

## 

- We could also estimate the model using only 1971 data.
  What would this be showing us?


In [ ]:
*| classes: larger
regress rprice nearinc if year == 1978, robust

##

- So, using 1981 data we capture the Total price difference between houses near and far from the incinerator. 
  - This captures both the announcement effect of the project, but also other factors (where would an incinerator be built?).
- Using 1978 data we capture the price difference between houses near and far from the incinerator in the absence of the project. 
  - This captures the effect of other factors that may be correlated with the incinerator project.
- Use both to see the impact!

$$Effect = -30688.27-(-18824.37)= -11863.9$$

- This is in essence a DnD model

## Difference in Differences


|  | Control| Treatment | Treat-Control |
|---|---|---|---|
| Pre-            | $\bar y_{00}$ | $\bar y_{10}$| $\bar y_{10}$-$\bar y_{00}$ |
| Post-           | $\bar y_{01}$ | $\bar y_{11}$ | $\bar y_{10}$-$\bar y_{00}$ |
| Post-pre        | $\bar y_{01}$-$\bar y_{00}$ | $\bar y_{11}$-$\bar y_{10}$ | DD  |
  
- Post-Pre: 
  - Trend changes for the control
  - Trend changes for the treated: A mix of the impact of the treatment and the trend change.
- Treat-Control: 
  - Baseline difference when looking at Pre-period
  - Total Price differentials when looking at Post-period: Mix of the impact of the treatment and the baseline difference.

- Take the Double Difference and you get the **treatment effect**.

## Difference in Differences: Regression {.scrollable}

- This could also be achieved using a regression model:

$$ y = \beta_0 + \beta_1 post + \beta_2 treat + \beta_3 post*treat + e$$

Where $\beta_3$ is the treatment effect. (only for 2x2 DD)


In [ ]:
regress rprice nearinc##y81, robust

## Difference in Differences: Regression + controls 

- One advantage of DD is that it can control for those unobserved factors that may be correlated with outcome. 
  - Without controls, however, estimates may not have enough precision.
- But, we could add controls!

$$ y = \beta_0 + X \gamma + \beta_1 post + \beta_2 treat + \beta_3 post*treat + e$$

- But its not as easy as it may seem! (just adding regressions is not a good approach)

- This method requires other assumptions! ($\gamma$ is fixed), which may be very strong.


>[**Note:**]{.redtxt} For DD to work, you need to assume the two groups follow the same path in the absence of the treatment. (Parallel trends assumption)
>
>Otherwise, you are just using trend differences!

## Diff in Diff in Diff

An Alternative approach is to use a triple difference model.

Setup:

- You still have two groups: Control and Treatment (which are easily identifiable)
- You have two time periods: Pre and Post (which are also easily identifiable)
- You have a different sample, where you can identify controls and treatment, as well as the pre- and post- periods. This sample was not treated!

Estimation: 

- Estimate the DD for the Original Sample, and the new untreated sample. 
- Obtaining the difference between these two estimates will give you the triple difference.

Example: Smoking ban analysis based on age. (DD) But using both treated and untreated States (DDD)

## General Framework and Pseudo Panels

- One general Structure for Policy analysis is the use of Pseudo Panels structure.
  - Pseudo panels are a way to use repeated crossection data, but controlling for some unobserved heterogeneity across specific groups. (the pseudo panels)
- For Pseudo-panels, we need to identify a group that could be followed across time. 
  - This cannot be a group of individuals (repeated crosection). 
  - But we could use groups of states, cohorts (year of birth), etc.
- In this case, the data would look like this:
$$y_{igt} = \lambda_t + \alpha_g + \beta x_{gt} + z_{igt}\gamma +  e_{igt}$$

- Where $g$ is the group, $t$ is the time, and $i$ is the individual.
- This model can be estimated by using dummies. (one dummy for each group and time-period)
- And $\beta$ is the coefficient of interest. (impact of the Policy $x_{gt}$).
  - This may ony work if we assume $\beta$ is constant across time and groups.

## Alternative

- We could also use a more general model:
$$y_{igt} = \lambda_{gt}+ \beta x_{gt} + z_{igt}\gamma +  e_{igt}$$

- where $\lambda_{gt}$ is a group-time fixed effect. (Dummy for each group-time combination) 
  - Nevertheless, while more flexible, this also imposes other types of assumptions, and might even be unfeasible if we have a large number of groups and time periods.

- Still, we require $\beta$ to be homogenous. If that is not the case, you may still suffer from contamination bias.

# Panel data
Baby steps: 2 period panel data

## 2-period Panel data

- Panel Data, or longitudinal data, is a type of data that has information about the same individual across time. 

- The simplest Structure is one where individuals are followed over only 2 periods. 
  
- The main advantage of panel data (even two periods version) is that it allows us to control for unobserved heterogeneity across individuals.
  - But only if you want to assume fixed effects are constant across time.

- Main caveat. For the methods we will see next, you cannot esitmate the effect of variables that are time fixed (or have little if any variation across time).
- 
## 

- So how does this reflects in the model specification?

$$y_{it} = \beta_0 + \beta_1 x_{it} + \beta_2 z_{t} + \beta_3 w_{i} + e_i + e_t + e_{it}$$

- Where $i$ refers to individuals or panel units, and $t$ refers to time periods.
- Also, $X's$, $X's$ $W's$ are variables that vary across individual and time, across time or across individuals.
- There are also three types of errors. Those that contains unobserved that vary across individuals $e_i$, across time $e_t$, and across individuals and time $e_{it}$ (Idiosyncratic error).
  
- $e_i$ is usually referred to as the individual fixed effect, and $e_t$ as the time fixed effect.

- In a 2 period panel, controlling for time-effects is may not be necessary (its just one dummy)

- What is more concerning is the unobserved individual fixed effect. 
 
This is pretty similar to the generalized Pooling model we saw before.

## How estimation changes

For time use, we assume we control with a single dummy.

1. You can choose to "ignore" individual effects. 

$$y_{it} = \beta_0 + \beta_1 x_{it} + \beta_2 w_{i} + \delta t + \underbrace{e_i + e_{it}}_{v_{it}}$$

   - Requires $e_i$ to be uncorrelated with $x_{it}$ (otherwise is biased), and Standard Errors will need to be clustered at the individual level.

2. You can aim to estimate **all** individual fixed effects using dummies (FE estimator).
$$y_{it} = \beta_0 + \beta_1 x_{it} + \delta t + \sum \alpha_i D_i + e_{it})$$

   - Time fixed variables cannot be estimated anymore
  
## 

3. You can estimate the model in differences (FD estimator)

$$\begin{aligned}
y_{i1} &= \beta_0 + \beta_1 x_{i1} + \delta + e_i + e_{i1} \\
y_{i0} &= \beta_0 + \beta_1 x_{i0} + e_i + e_{i0} \\
\Delta y_{i} &= \ \ \ \ \ \ \ \ \ \beta_1 \Delta x_{i1} + \delta + \Delta e_{i}
\end{aligned}
$$

- Now you have only 1 observation per panel, instead of 2. And the result would be identical to FE estimator.

## Example {.scrollable}


In [ ]:
** This data is in wide format
frause slp75_81, clear
** Lets reshape it so its in standard long format
qui:gen id = _n
qui:reshape long educ gdhlth marr slpnap totwrk yngkid, i(id) j(year)
qui:gen dtime = year==81
qui:xtset id dtime
** Regression as Pool Crossection
qui: reg slpnap dtime totwrk educ marr yngkid gdhlth male,  
local cname:colnames e(b)
display "`cname'"
est sto m1
** using FE
qui: areg slpnap dtime totwrk educ marr yngkid gdhlth male, absorb(id)  
est sto m2
** using FD
qui: reg d.slpnap d.dtime d.totwrk d.educ d.marr d.yngkid d.gdhlth d.male, 
matrix b = e(b)
matrix V = e(V)
matrix colname b = `cname'
matrix colname V = `cname'
matrix rowname V = `cname'
adde repost b=b V=V, rename
est sto m3
esttab m1 m2 m3, se r2

## Nx2 DID with Panel 

- Using panel data also allows you to analyze policies using canonical DID approach (2x2).
  - It simplifies the process because one of the differences can be estiamated right away.
- Consider the Standard DID Model:
$$y_{it} = a_0 + a_1 post + \color{red}{a_2 treat} + \color{green}{a_3 post \times treat}+ e_{it} $$

- With panel data, you can extend this allowing for "multiple groups", and controling for whether Received treatment or not: 

$$y_{it} = a_0 + a_1 dtime + \color{red}{a_i} + \color{green}{a_3 treated} + e_{it} $$
$$\Delta y_{i} =  a_1 + \color{green}{a_3 treated} + \Delta e_{i} $$

- Most effective way to control for unobserved individual effects!
  
##

:::{.callout-warning}

## Beware of the TWFE

**warning** you may be tempted to say, with more time-periods, just add more dummies. But that would be wrong!

Look for all the new literature on DID with Multiple periods and treatment timing

:::

## Panel Data with more than 2 periods

- So far we have covered the case when you observe individuals for two periods only. What if you observe them for more than two periods?

- The model is pretty much the same.
  - You will mostly need to add an additional time dummy.
  - You also have multiple approaches that would allow you to estimate the model.
  - Adding Dummies still work.
  - And Difference in Differences still work.
  - You can also choose Randome-effect model (next class)

## Differencing

- Consider the following model:

$$y_{it} = \beta_0 + \beta_1 x_{it} +a_i + \delta_2 T_2 + \delta_3 T_3 + e_{it}$$

- Where $T_2$ and $T_3$ are time dummies for the second and third periods. If apply a first differences transformation, we get:

$$\Delta y_{it} = \beta_1 \Delta x_{it} + \delta_2 + \delta_3 T_3 + \Delta e_{it}$$

## 

- Mechanically, its the same as before. However,analytically, this imposes stronger assumptions

$$Corr(\Delta x_{it}, \Delta e_{it}) =Corr(x_{it}-x_{it-1}, e_{it}-e_{it-1}) = 0$$

- This is a stronger assumption than $Corr(x_{it}, e_{it}) = 0$. It implies that $x_{it}$ has to be strictly exogenous to errors $e_{it}$, regardless of timing.

- One may want to also consider using Other Standard error corrections.

## FD is to be used with caution

- FD is an easy estimation with panel data. However, it should be used with caution
  - It depends much more on the strict exogeneity assumption. (may create biases)
  - If one adds Lagged Dep variables as controls, further problems may arise. 
  - Issues with measurement error are magnified when using First Differences as well. (and can be difficult to address)

- But, there are still other methods that could be used to estimate panel data models with multiple periods:
  - FE (dummy inclusion approach)
  - Random Effects (next class)
  - Correlated Random Effects (next class)

# {background-image="https://i.imgflip.com/7zjmz3.jpg" background-size="contain"}

# Thats all for today 
Next class...Advance Panel Data Methods